In [190]:
import pandas as pd
import numpy as np
from sklearn.tree import export_graphviz
import pydot
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import model_selection, metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [191]:
b = pd.read_csv("2_KS_train_w_vader.csv")

In [192]:
df = pd.read_csv("KS_train_final.csv")

In [193]:
df['blurb'] = b['blurb']

In [194]:
df['blurb'].fillna(" ", inplace=True)

In [195]:
for index, value in df['blurb'].items():
    s = "".join([ c if c.isalpha() else " " for c in value ])
    s = s.lower()
    
    df.at[index, 'clean_blurb'] = s

In [196]:
df = df.drop(['blurb'], axis=1)

In [197]:
df.head(5)

,staff_pick,funded,is_cat_art,is_cat_music,is_cat_film,is_cat_technology,is_cat_publishing,is_cat_food,is_cat_games,is_cat_fashion,...,is_2017,is_2018,sentiment_pos,sentiment_neu,sentiment_neg,blurb_bayes,log_goal_usd,log_days_to_launch,log_days_to_dealine,clean_blurb
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.770405,0.536852,0.571877,0.636119,i will be an artist in residence at elsewhere ...
1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.913929,0.557160,0.417726,0.786681,artists is a public art making installati...
2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.660800,0.561670,0.269857,0.890160,the sequel to my favorite machine myphoneh...
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.993950,0.572900,0.285142,0.624541,a film exploring the role and value of art edu...
4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.623790,0.608693,0.515240,0.713862,we need to build a kitchen for habitable space...


In [198]:
X = df.drop(['funded', 'is_cat_music', 'is_cat_film', 'is_cat_publishing', 'is_cat_games', 'is_cat_fashion',
            'is_cat_comics', 'is_cat_other', 'is_country_us', 'is_loc_ca', 'is_loc_ny', 'is_loc_uk',
            'is_loc_tx', 'is_loc_cd', 'is_loc_fl', 'is_loc_il', 'is_loc_wa', 'is_loc_pa',
            'is_loc_other', 'is_2009', 'is_2017', 'is_2018', 'blurb_bayes', 'sentiment_pos', 'sentiment_neu',
            'sentiment_neg'], axis=1)

In [199]:
y = df['funded']

In [200]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [201]:
X_NB_train = X_train.clean_blurb

In [202]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(X_NB_train.values)

In [203]:
classifier = MultinomialNB(alpha=1)
targets = y_train.values
classifier.fit(counts, targets)

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

In [204]:
predictions = classifier.predict_proba(counts)
# i = 0

# for row in predictions:
#     X_train.loc[i, 'blurb_bayes'] = row[1]
#     i+=1

In [205]:
X_train['blurb_bayes'] = predictions.T[1]

c:\users\ajay mota\pycharmprojects\bit\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [206]:
X_train = X_train.drop(['clean_blurb'], axis=1)

In [207]:
X_train.head(5)

,staff_pick,is_cat_art,is_cat_technology,is_cat_food,is_2010,is_2011,is_2012,is_2013,is_2014,is_2015,is_2016,log_goal_usd,log_days_to_launch,log_days_to_dealine,blurb_bayes
2782,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.483708,0.232524,0.557388,0.077252
65081,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.561670,0.505056,0.799086,0.990496
16706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.542600,0.000000,0.657836,0.991201
33555,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.660226,0.208863,0.713862,0.465348
78185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.561670,0.208863,0.713862,0.401861


In [208]:
pd.read_csv("KS_train_final.csv").iloc[[78185]]

,staff_pick,funded,is_cat_art,is_cat_music,is_cat_film,is_cat_technology,is_cat_publishing,is_cat_food,is_cat_games,is_cat_fashion,...,is_2016,is_2017,is_2018,sentiment_pos,sentiment_neu,sentiment_neg,blurb_bayes,log_goal_usd,log_days_to_launch,log_days_to_dealine
78185,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.378554,0.56167,0.208863,0.713862


In [209]:
X_train['blurb_bayes'].isna().sum() 

0

In [210]:
rfc = RandomForestClassifier(n_estimators=10, max_depth=10, max_features='sqrt', random_state = 42)
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [211]:
####### NAIVE BAYES on test data #########

counts = vectorizer.transform(X_test.clean_blurb.values)
predictions = classifier.predict_proba(counts)

In [212]:
X_test['blurb_bayes'] = predictions.T[1]
X_test = X_test.drop(['clean_blurb'], axis=1)

c:\users\ajay mota\pycharmprojects\bit\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [218]:
rfc_predict = rfc.predict(X_test)

In [219]:
rfc_cv_score = cross_val_score(rfc, X_test, y_test, cv=10, scoring='roc_auc')

In [220]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[ 6801  3512]
 [ 2521 11075]]


=== Classification Report ===
              precision    recall  f1-score   support

         0.0       0.73      0.66      0.69     10313
         1.0       0.76      0.81      0.79     13596

    accuracy                           0.75     23909
   macro avg       0.74      0.74      0.74     23909
weighted avg       0.75      0.75      0.75     23909



=== All AUC Scores ===
[0.82712893 0.84056408 0.83440635 0.83951582 0.83882082 0.84076496
 0.8401771  0.84430698 0.82696775 0.8343261 ]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.836697890534675


In [221]:
rfc_predict = rfc.predict(X_train)

In [224]:
rfc_cv_score = cross_val_score(rfc, X_train, y_train, cv=10, scoring='roc_auc')

In [225]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_train, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_train, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[23365  7656]
 [ 5265 35439]]


=== Classification Report ===
              precision    recall  f1-score   support

         0.0       0.82      0.75      0.78     31021
         1.0       0.82      0.87      0.85     40704

    accuracy                           0.82     71725
   macro avg       0.82      0.81      0.81     71725
weighted avg       0.82      0.82      0.82     71725



=== All AUC Scores ===
[0.88626014 0.89076833 0.8938258  0.88776138 0.8943784  0.88630479
 0.88782604 0.89266729 0.88828298 0.88708676]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.8895161892005877
